#Objetivo
Realizar um fluxo de dados entre dois buckets (s3) utilizando Python


#Exercicio 1: 
###Buscar dados do bucket chamado-tecnico-ac3 e gravar no bucket

In [ ]:
%%sh
sudo pip install pyodbc
sudo pip install sqlalchemy
sudo pip install pymysql
sudo pip install mysqlclient
sudo pip install boto3
sudo pip install s3fs

In [ ]:
#Importar bibliotecas para ambiente de trabalho
import sqlalchemy, pyodbc, boto3, s3fs, pandas as pd

In [ ]:
#Credenciais acesso ao aws 
aws_access_key_id_prof = ''
aws_secret_access_key_prof = ''
aws_session_token_prof = ''

In [ ]:
#Criar recurso aws prof
resource_s3_prof = boto3.resource('s3', aws_access_key_id = aws_access_key_id_prof,
                       aws_secret_access_key = aws_secret_access_key_prof,
                       aws_session_token = aws_session_token_prof, use_ssl=False)

bucket_professor = resource_s3_prof.Bucket("chamado-tecnico-ac3")

In [ ]:
#Subir um arquivo de exemplo
bucket_professor.download_file(Key="chamados/chamado.csv",Filename="chamados/chamado.csv")

In [ ]:
#Credenciais de acesso ao bucket de destino
aws_access_key_id_destino = ''
aws_secret_access_key_destino = ''
aws_session_token_destino = ''

In [ ]:
#cliente para criar um novo bucket
client_s3 = boto3.client('s3',
                      aws_access_key_id = aws_access_key_id_destino,
                      aws_secret_access_key = aws_secret_access_key_destino,
                      aws_session_token = aws_session_token_destino, use_ssl=False)

nome_bucket = "chamado-tecnico-1201555"
client_s3.create_bucket(Bucket=nome_bucket)

In [ ]:
#Criar recurso aws bucket destino
resource_s3_prof = boto3.resource('s3', aws_access_key_id = aws_access_key_id_destino,
                       aws_secret_access_key = aws_secret_access_key_destino,
                       aws_session_token = aws_session_token_destino, use_ssl=False)

bucket_destino = resource_s3_prof.Bucket("chamado-tecnico-1201555")

In [ ]:
#subir dados para bucket destino manual
bucket_destino.upload_file(Key="chamados/chamado.csv",Filename="chamados/chamado.csv")

In [ ]:
#cliente banco professor, para buscar metadados
client_s3_prof = boto3.client('s3',
                      aws_access_key_id = aws_access_key_id_prof,
                      aws_secret_access_key = aws_secret_access_key_prof,
                      aws_session_token = aws_session_token_prof, use_ssl=False)
client_s3_prof.list_buckets()

In [ ]:
#Cria lista com contents
contents = client_s3_prof.list_objects(Bucket = 'chamado-tecnico-ac3').get("Contents")
#Filtra arquivos com .csv em uma nova lista
lista_arquivos = [i.get("Key") for i in contents if ".csv" in i.get("Key")]

#grava dados do bucket professor na pasta intermediario e depois da pasta intermediario para destino
for i in lista_arquivos:
  arquivo = i
  diretorio = i.replace("chamados","intermediario")
  print(arquivo,diretorio)
  bucket_professor.download_file(Key=arquivo, Filename=diretorio)
  bucket_destino.upload_file(Key=arquivo, Filename=diretorio)

chamados/categoria.csv intermediario/categoria.csv
chamados/chamado.csv intermediario/chamado.csv
chamados/cidade.csv intermediario/cidade.csv
chamados/clientes.csv intermediario/clientes.csv
chamados/equipes.csv intermediario/equipes.csv
chamados/estado.csv intermediario/estado.csv
chamados/setor.csv intermediario/setor.csv
chamados/tipo.csv intermediario/tipo.csv
chamados/unidade.csv intermediario/unidade.csv
chamados/venda.csv intermediario/venda.csv


In [ ]:
def troca_arquivos_bucket(bucket_fonte,bucket_destino,arquivo):
  bucket_fonte.download_file(Key=arquivo, Filename=arquivo)
  bucket_destino.upload_file(Key=arquivo, Filename=arquivo)

In [ ]:
#Usando uma função para modularizar o codigo
#Cria lista com contents
contents = client_s3_prof.list_objects(Bucket = 'chamado-tecnico-ac3').get("Contents")
#Filtra arquivos com .csv em uma nova lista
lista_arquivos = [i.get("Key") for i in contents if ".csv" in i.get("Key")]

#grava dados do bucket professor na pasta intermediario e depois da pasta intermediario para destino
for i in lista_arquivos:
  arquivo = i   
  troca_arquivos_bucket(bucket_fonte = bucket_professor, bucket_destino=bucket_destino,arquivo=i)

In [ ]:
[troca_arquivos_bucket(bucket_fonte = bucket_professor, bucket_destino=bucket_destino,arquivo=i.get("Key")) for i in contents if ".csv" in i.get("Key")]   

Usando o s3fs


In [ ]:
#Subindo manualmente
fs_prof = s3fs.S3FileSystem(key=aws_access_key_id_prof,
                       secret=aws_secret_access_key_prof,
                       token =aws_session_token_prof)

#Criando um objeto do tipo s3fs.
fs_dest = s3fs.S3FileSystem(key=aws_access_key_id_destino,
                       secret=aws_secret_access_key_destino,
                       token =aws_session_token_destino)

In [ ]:
#Subindo manualmente
arquivo = fs_prof.open("chamado-tecnico-ac3/chamados/chamado.csv")
df = pd.read_csv(arquivo,delimiter=';')
with fs_dest.open("chamado-tecnico-1201555/chamados_2/chamado.csv",mode="w") as f:
  df.to_csv(f)


['chamado-tecnico-ac3/chamados/categoria.csv',
 'chamado-tecnico-ac3/chamados/chamado.csv',
 'chamado-tecnico-ac3/chamados/cidade.csv',
 'chamado-tecnico-ac3/chamados/clientes.csv',
 'chamado-tecnico-ac3/chamados/equipes.csv',
 'chamado-tecnico-ac3/chamados/estado.csv',
 'chamado-tecnico-ac3/chamados/setor.csv',
 'chamado-tecnico-ac3/chamados/tipo.csv',
 'chamado-tecnico-ac3/chamados/unidade.csv',
 'chamado-tecnico-ac3/chamados/venda.csv']

In [ ]:
#Subind dinamico

#cria a lista de arquivos
contents = fs_prof.listdir("chamado-tecnico-ac3/chamados")
lista_arquivos = [i.get("Key") for i in contents if ".csv" in i.get("Key")]
for i  in lista_arquivos:
  arquivo = fs_prof.open(i)
  df = pd.read_csv(arquivo,delimiter=';')
  with fs_dest.open(i,mode="w") as f:
    df.to_csv(f)





#Exercicio 2: 
1. Criar um bucket chamado "bucket-maquinas-{RA DO ALUNO}"
2. Enviar as tabelas do banco de dados MAQUINA Para o bucket no S3 chamado bucket-maquinas-{RA DO ALUNO} 



In [ ]:
server = ''
database = "MAQUINA"
username ='admin'
password = 'aluno123'
stringbound = f'mysql://{username}:{password}@{server}/{database}'
engine=sqlalchemy.create_engine(stringbound)
engine.connect()

In [ ]:
nome_bucket = "maquina-1201555"
client_s3.create_bucket(Bucket=nome_bucket)

In [ ]:
#buscar nome das tabelas para integrar
df_tabela = pd.read_sql("SELECT TABLE_SCHEMA,TABLE_NAME FROM information_schema.tables where TABLE_SCHEMA IN ('MAQUINA') ;", con=engine)
df_tabela

,TABLE_SCHEMA,TABLE_NAME
0,MAQUINA,CIDADE
1,MAQUINA,CLIENTE
2,MAQUINA,ENDERECO
3,MAQUINA,ESTADO
4,MAQUINA,ITEM_PEDIDO
5,MAQUINA,MAQUINA
6,MAQUINA,PEDIDO


In [ ]:
#Subindo dados do banco para o bucket
for x,y in df_tabela.iterrows():
  tabela = y['TABLE_NAME']
  df = pd.read_sql(tabela,con=engine)
  caminho_bucket = f'maquina-1201555/maquina/{tabela.lower()}.csv'
  with fs_dest.open(caminho_bucket,mode="w") as f:
    df.to_csv(f)
 

In [ ]:
for x,y in df_tabela.iterrows():
  tabela = y['TABLE_NAME']
  banco = y['TABLE_SCHEMA']
  df = pd.read_sql(f"select * from {banco}.{tabela}",con=engine)
  caminho_bucket = f'maquina-1201555/{banco}/{tabela.lower()}.csv'
  with fs_dest.open(caminho_bucket,mode="w") as f:
    df.to_csv(f)
 

Códigos para consulta

In [ ]:
with open("chamados/chamado_teste.csv",mode="w") as f:
   df.to_csv(f)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
arquivos = [i.get("Key") for i in contents]
arquivos

['anscombe',
 'chamados',
 'chamados/',
 'chamados/categoria.csv',
 'chamados/chamado.csv',
 'chamados/cidade.csv',
 'chamados/clientes.csv',
 'chamados/equipes.csv',
 'chamados/estado.csv',
 'chamados/setor.csv',
 'chamados/tipo.csv',
 'chamados/unidade.csv',
 'chamados/venda.csv']

In [ ]:
for i in contents:
  arquivo = i.get("Key")
  print(arquivo)

anscombe
chamados
chamados/
chamados/categoria.csv
chamados/chamado.csv
chamados/cidade.csv
chamados/clientes.csv
chamados/equipes.csv
chamados/estado.csv
chamados/setor.csv
chamados/tipo.csv
chamados/unidade.csv
chamados/venda.csv


In [ ]:
lista2 = []
for i in contents:
  arquivo = i.get("Key")
  if ".csv" in arquivo:        
    lista2.append(arquivo)
lista2

['chamados/categoria.csv',
 'chamados/chamado.csv',
 'chamados/cidade.csv',
 'chamados/clientes.csv',
 'chamados/equipes.csv',
 'chamados/estado.csv',
 'chamados/setor.csv',
 'chamados/tipo.csv',
 'chamados/unidade.csv',
 'chamados/venda.csv']

In [ ]:
lista2.append("chamados/gato.csv")
lista2

['anscombe',
 'chamados',
 'chamados/',
 'chamados/categoria.csv',
 'chamados/chamado.csv',
 'chamados/cidade.csv',
 'chamados/clientes.csv',
 'chamados/equipes.csv',
 'chamados/estado.csv',
 'chamados/setor.csv',
 'chamados/tipo.csv',
 'chamados/unidade.csv',
 'chamados/venda.csv',
 'chamados/gato.csv']

In [ ]:
{dicionario = {"Key":"chamados/categoria.csv"}

In [ ]:
dicionario.get("Key")

'chamados/categoria.csv'

In [ ]:
#abre a lista e insere em outra lista
lista_c = [i for i in contents]

#aplica um metodo nos objetos da lista
lista_d = [i.get("Key") for i in contents]

#aplicando um filtro na lista
lista_e = [i.get("Key") for i in contents if ".csv" in i.get("Key")]
lista_e

['chamados/categoria.csv',
 'chamados/chamado.csv',
 'chamados/cidade.csv',
 'chamados/clientes.csv',
 'chamados/equipes.csv',
 'chamados/estado.csv',
 'chamados/setor.csv',
 'chamados/tipo.csv',
 'chamados/unidade.csv',
 'chamados/venda.csv']